 # Lab 5



 ## Background



 The Deutsch-Jozsa algorithm demonstrates quantum advantage by determining whether a black-box function is constant or balanced using only **one quantum query**, compared to up to $2^{n - 1} + 1$ classical queries.



 **Promise**: The function $f: \{0,1\}^n → \{0,1\}$ is either:



 - **Constant**: Same output for all inputs (all 0s or all 1s)

 - **Balanced**: Exactly half the outputs are 0, half are 1

 ## Imports



 Run this first to import all necessary libraries:

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import numpy as np

 ## Part 1: Quantum Circuit Basics and Setup (Revision)

 ### Exercise 1.1: Basic Quantum Circuit Operations



 Let's start by doing a quick revision of Qiskit and basic quantum operations:

In [2]:
def create_basic_circuit():
    """
    Create a 2-qubit circuit that:
    1. Puts the first qubit into the |1⟩ state.
    2. Puts the second qubit into a superposition state.
    3. Adds a barrier to the circuit.
    4. Measures both qubits.
    
    Returns:
        QuantumCircuit: A 2-qubit quantum circuit.
    """
    qc = QuantumCircuit(2, 2)
    qc.x(0)
    qc.h(1)
    qc.barrier()
    qc.measure(range(2),range(2))
    # Put qubit 0 in the |1⟩ state.
    
    # Put qubit 1 in a superposition state.
    
    # Add a barrier.
    
    # Measure both qubits.
    
    return qc

# Test your implementation
circuit = create_basic_circuit()
print(circuit.draw())

# Run simulation and analyze results
simulator = AerSimulator()
# Simulate the circuit and print measurement results
job = simulator.run(circuit, shots=1024)
result = job.result()
counts = result.get_counts()
print("\nMeasurement results:", counts)


     ┌───┐ ░ ┌─┐   
q_0: ┤ X ├─░─┤M├───
     ├───┤ ░ └╥┘┌─┐
q_1: ┤ H ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 

Measurement results: {'01': 525, '11': 499}


 ### Exercise 1.2: Understanding Quantum States



 Practice creating different quantum states:

In [3]:
def create_plus_state():
    """
    Creates a circuit that prepares the |+⟩ state.
    
    The |+⟩ state is a superposition state, equal to (|0⟩ + |1⟩)/√2.
    
    Returns:
        QuantumCircuit: A 1-qubit circuit preparing the |+⟩ state.
    """
    qc = QuantumCircuit(1, 1)
    # Apply the appropriate operation to prepare the |+⟩ state.
    # Measure the qubit.
    qc.h(0)
    qc.measure(0,0)
    return qc

def create_minus_state():
    """
    Creates a circuit that prepares the |−⟩ state.
    
    The |−⟩ state is a superposition state, equal to (|0⟩ - |1⟩)/√2.
    
    Returns:
        QuantumCircuit: A 1-qubit circuit preparing the |−⟩ state.
    """
    qc = QuantumCircuit(1, 1)
    # Apply the required operations to prepare the |−⟩ state.
    # Measure the qubit.
    qc.h(0)
    qc.s(0)
    qc.s(0)
    qc.measure(0,0)
    return qc

def create_bell_state():
    """
    Creates a circuit that prepares the Bell state (|00⟩ + |11⟩)/√2.
    
    This is a maximally entangled two-qubit state.
    
    Returns:
        QuantumCircuit: A 2-qubit circuit preparing a Bell state.
    """
    qc = QuantumCircuit(2, 2)
    # Apply the operations to create the entangled state.
    # Measure both qubits.
    qc.h(0)
    qc.cx(0,1)
    qc.measure(range(2),range(2))
    return qc


In [4]:
# Test each state
states = {
    'Plus State': create_plus_state(),
    'Minus State': create_minus_state(),
    'Bell State': create_bell_state()
}

for name, circuit in states.items():
    print(f"\n{name}:")
    print(circuit.draw())
    job = simulator.run(circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    print("Results:", counts)
    



Plus State:
     ┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 
Results: {'0': 506, '1': 518}

Minus State:
     ┌───┐┌───┐┌───┐┌─┐
  q: ┤ H ├┤ S ├┤ S ├┤M├
     └───┘└───┘└───┘└╥┘
c: 1/════════════════╩═
                     0 
Results: {'0': 475, '1': 549}

Bell State:
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 
Results: {'00': 500, '11': 524}


 ### Exercise 1.3: Simple Oracle Concept



 Understand what an oracle does:

In [5]:
def create_constant_oracle(output):
    """
    Creates an oracle for a constant function f(x) = c.
    
    The oracle is a unitary operation that maps $|x⟩|y⟩$ to $|x⟩|y \oplus f(x)⟩$.
    If the function is constant with a non-zero output, a specific gate must be applied.
    
    Args:
        output (int): The constant output of the function, either 0 or 1.
        
    Returns:
        QuantumCircuit: A 2-qubit circuit representing the oracle.
    """
    oracle = QuantumCircuit(2, name=f'f(x)={output}')
    if output == 1:
        # Apply the required operation to the ancilla (qubit 1).
        oracle.x(1)
    
    return oracle

# Test both oracles
oracle_0 = create_constant_oracle(0)
oracle_1 = create_constant_oracle(1)

print("Constant-0 Oracle:")
print(oracle_0.draw())
print("\nConstant-1 Oracle:")  
print(oracle_1.draw())


Constant-0 Oracle:
     
q_0: 
     
q_1: 
     

Constant-1 Oracle:
          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘


<>:5: SyntaxWarning: invalid escape sequence '\o'
<>:5: SyntaxWarning: invalid escape sequence '\o'
C:\Users\PaccoTan\AppData\Local\Temp\ipykernel_26040\895320350.py:5: SyntaxWarning: invalid escape sequence '\o'
  The oracle is a unitary operation that maps $|x⟩|y⟩$ to $|x⟩|y \oplus f(x)⟩$.


 ## Part 2: Single-Qubit Deutsch Algorithm

 ### Exercise 2.1: Implement Four Oracle Functions

In [6]:
def create_constant_0_oracle():
    """
    Creates an oracle for the constant function f(x) = 0.
    
    Returns:
        QuantumCircuit: An empty 2-qubit circuit.
    """
    oracle = QuantumCircuit(2, name='Constant-0')
    return oracle

def create_constant_1_oracle():
    """
    Creates an oracle for the constant function f(x) = 1.
    
    Returns:
        QuantumCircuit: A 2-qubit circuit with a gate on the ancilla.
    """  
    oracle = QuantumCircuit(2, name='Constant-1')
    oracle.x(1)
    # Apply the operation to the ancilla qubit.
    return oracle

def create_balanced_identity_oracle():
    """
    Creates an oracle for the balanced function f(x) = x.
    
    Returns:
        QuantumCircuit: A 2-qubit circuit with a controlled gate.
    """
    oracle = QuantumCircuit(2, name='Balanced-Identity')
    
    # Apply the appropriate controlled operation.
    oracle.cx(0,1)
    return oracle

def create_balanced_not_oracle():
    """
    Creates an oracle for the balanced function f(x) = 1⊕x.
    
    Returns:
        QuantumCircuit: A 2-qubit circuit with two gates.
    """
    oracle = QuantumCircuit(2, name='Balanced-NOT')
    # Apply the required operations to the qubits.
    oracle.cx(0,1)
    oracle.x(1)
    return oracle


In [7]:
# Test your oracles by visualizing them
oracles = {
    'Constant 0': create_constant_0_oracle(),
    'Constant 1': create_constant_1_oracle(), 
    'Balanced Identity': create_balanced_identity_oracle(),
    'Balanced NOT': create_balanced_not_oracle()
}

for name, oracle in oracles.items():
    print(f"{name}:")
    print(oracle.draw())
    print()


Constant 0:
     
q_0: 
     
q_1: 
     

Constant 1:
          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

Balanced Identity:
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

Balanced NOT:
               
q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘



 ### Exercise 2.2: Implement Deutsch Algorithm

In [8]:
def deutsch_algorithm(oracle):
    """
    Implements the full Deutsch algorithm for a single input qubit (n=1).
    
    The algorithm uses two qubits: an input qubit and an ancilla qubit.
    1. Initialize the ancilla to a specific state.
    2. Apply a superposition-creating gate to both qubits.
    3. Apply the oracle.
    4. Apply a final superposition-creating gate to the input qubit.
    5. Measure the input qubit.
    
    Args:
        oracle (QuantumCircuit): The oracle circuit representing the function.
        
    Returns:
        QuantumCircuit: The complete circuit for the Deutsch algorithm.
    """
    qc = QuantumCircuit(2, 1)
    
    # Step 1: Initialize the ancilla.
    qc.x(1)
    qc.barrier()
    # Step 2: Apply the superposition-creating gate to both qubits.
    qc.h(range(2))
    qc.barrier()
    # Step 3: Apply the oracle.
    qc = qc.compose(oracle)
    qc.barrier()
    # Step 4: Apply a final superposition-creating gate to the input qubit only.
    qc.h(0)
    qc.barrier()
    # Step 5: Measure the input qubit.
    qc.measure(0,0)
    return qc

# Visualize the complete algorithm with one oracle
sample_oracle = create_balanced_identity_oracle()
complete_circuit = deutsch_algorithm(sample_oracle)
print("Complete Deutsch Algorithm:")
print(complete_circuit.draw())


Complete Deutsch Algorithm:
           ░ ┌───┐ ░       ░ ┌───┐ ░ ┌─┐
q_0: ──────░─┤ H ├─░───■───░─┤ H ├─░─┤M├
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐ ░ └───┘ ░ └╥┘
q_1: ┤ X ├─░─┤ H ├─░─┤ X ├─░───────░──╫─
     └───┘ ░ └───┘ ░ └───┘ ░       ░  ║ 
c: 1/═════════════════════════════════╩═
                                      0 


In [9]:
def test_deutsch_algorithm():
    """Test Deutsch algorithm with all four oracles"""
    
    oracles = {
        'Constant 0': create_constant_0_oracle(),
        'Constant 1': create_constant_1_oracle(),
        'Balanced (x)': create_balanced_identity_oracle(),
        'Balanced (1⊕x)': create_balanced_not_oracle()
    }
    
    print("Deutsch Algorithm Test Results:")
    print("=" * 50)
    
    for name, oracle in oracles.items():
        circuit = deutsch_algorithm(oracle)
        
        job = simulator.run(circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()
        
        if '0' in counts and counts.get('0', 0) > 500:
            function_type = "CONSTANT"
        else:
            function_type = "BALANCED"
            
        print(f"{name}:")
        print(f"  Results: {counts}")
        print(f"  Type: {function_type}")
        print()


test_deutsch_algorithm()


Deutsch Algorithm Test Results:
Constant 0:
  Results: {'0': 1024}
  Type: CONSTANT

Constant 1:
  Results: {'0': 1024}
  Type: CONSTANT

Balanced (x):
  Results: {'1': 1024}
  Type: BALANCED

Balanced (1⊕x):
  Results: {'1': 1024}
  Type: BALANCED



 **Expected Results**:



 - Constant functions → Always measure '0'

 - Balanced functions → Always measure '1'

 ## Part 3: Multi-Qubit Deutsch-Jozsa Algorithm

 ### Exercise 3.1: Create General Oracle Function

In [10]:
def create_dj_oracle(n, function_type, mask=None):
    """
    Creates a Deutsch-Jozsa oracle for an n-qubit input.
    
    The oracle is an (n+1)-qubit circuit. The ancilla is the last qubit (n).
    
    - For constant functions, the oracle applies a gate to the ancilla
      if the output is 1, otherwise it does nothing.
    - For balanced functions, the oracle applies a controlled gate from each
      input qubit `i` to the ancilla if the `i`-th bit of the `mask` is '1'.
      This implements the inner product $f(x) = \sum_{i} x_i \cdot mask_i \pmod{2}$.
    
    Args:
        n (int): The number of input qubits.
        function_type (str): Either 'constant' or 'balanced'.
        mask (str, optional): A binary string used to define balanced functions or
                              the constant output for constant functions.
    
    Returns:
        QuantumCircuit: The oracle circuit.
    """
    oracle = QuantumCircuit(n + 1, name=f'DJ-{function_type}')
    
    if function_type == 'constant':
        if mask == '1': 
            # Apply the appropriate operation to the ancilla (qubit n).
            oracle.x(n)
            pass
    
    elif function_type == 'balanced':
        if mask:
            # Iterate through the mask and apply a controlled operation for each '1' bit.
            for i, bit in enumerate(mask):
                if bit == '1':
                    oracle.cx(i,n)
    
    return oracle

# Test the oracle creation
test_oracle = create_dj_oracle(3, 'balanced', '101')
print("3-qubit balanced oracle (mask='101'):")
print(test_oracle.draw())


3-qubit balanced oracle (mask='101'):
               
q_0: ──■───────
       │       
q_1: ──┼───────
       │       
q_2: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├
     └───┘└───┘


<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
C:\Users\PaccoTan\AppData\Local\Temp\ipykernel_26040\149676844.py:11: SyntaxWarning: invalid escape sequence '\s'
  This implements the inner product $f(x) = \sum_{i} x_i \cdot mask_i \pmod{2}$.


 ### Exercise 3.2: Implement Full Deutsch-Jozsa Algorithm

In [11]:
def deutsch_jozsa_algorithm(n, oracle):
    """
    Implements the full Deutsch-Jozsa algorithm for n input qubits.
    
    The circuit uses n input qubits and one ancilla qubit.
    1. Initialize the ancilla.
    2. Apply a superposition-creating gate to all qubits.
    3. Apply the oracle.
    4. Apply a final superposition-creating gate to the input qubits only.
    5. Measure the input qubits.
    
    Args:
        n (int): The number of input qubits.
        oracle (QuantumCircuit): The oracle circuit representing the function.
        
    Returns:
        QuantumCircuit: The complete Deutsch-Jozsa circuit.
    """
    qc = QuantumCircuit(n + 1, n)
    
    # Step 1: Initialize the ancilla.
    qc.x(n)
    qc.barrier()
    # Step 2: Apply the superposition-creating gate to all qubits.
    qc.h(range(n+1))
    qc.barrier()
    # Step 3: Apply the oracle.
    qc = qc.compose(oracle)
    qc.barrier()
    # Step 4: Apply the final superposition-creating gate to the input qubits.
    qc.h(range(n))
    qc.barrier()
    # Step 5: Measure the input qubits.
    qc.measure(range(n),range(n))
    return qc

# Test with the oracle from above
test_circuit = deutsch_jozsa_algorithm(3, test_oracle)
print("Complete 3-qubit Deutsch-Jozsa circuit:")
print(test_circuit.draw())


Complete 3-qubit Deutsch-Jozsa circuit:
           ░ ┌───┐ ░            ░ ┌───┐ ░ ┌─┐      
q_0: ──────░─┤ H ├─░───■────────░─┤ H ├─░─┤M├──────
           ░ ├───┤ ░   │        ░ ├───┤ ░ └╥┘┌─┐   
q_1: ──────░─┤ H ├─░───┼────────░─┤ H ├─░──╫─┤M├───
           ░ ├───┤ ░   │        ░ ├───┤ ░  ║ └╥┘┌─┐
q_2: ──────░─┤ H ├─░───┼────■───░─┤ H ├─░──╫──╫─┤M├
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐┌─┴─┐ ░ └───┘ ░  ║  ║ └╥┘
q_3: ┤ X ├─░─┤ H ├─░─┤ X ├┤ X ├─░───────░──╫──╫──╫─
     └───┘ ░ └───┘ ░ └───┘└───┘ ░       ░  ║  ║  ║ 
c: 3/══════════════════════════════════════╩══╩══╩═
                                           0  1  2 


 ### Comprehensive Testing

In [12]:
def test_deutsch_jozsa():
    """Test with multiple cases"""
    
    test_cases = [
        (2, 'constant', None, "Should measure 00"),
        (2, 'constant', '1', "Should measure 00"), 
        (3, 'balanced', '101', "Should measure 101"),
        (3, 'balanced', '111', "Should measure 111"),
        (2, 'balanced', '01', "Should measure 01"),
    ]
    
    print("Deutsch-Jozsa Test Results:")
    print("=" * 60)
    
    for n, func_type, mask, expected in test_cases:
        print(f"\nTest: n={n}, {func_type}, mask={mask}")
        print(f"Expected: {expected}")
        
        if mask is not None:
            mask = mask[::-1]
            
        oracle = create_dj_oracle(n, func_type, mask)
        circuit = deutsch_jozsa_algorithm(n, oracle)
        
        job = simulator.run(circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()
        
        most_frequent = max(counts.keys(), key=lambda x: counts[x])
        
        print(f"Results: {counts}")
        print(f"Most frequent: {most_frequent}")
        
        if func_type == 'constant':
            success = most_frequent == '0' * n
        else:
            success = most_frequent != '0' * n
            
        print(f"Correct: {success}")

# Run comprehensive test
test_deutsch_jozsa()

Deutsch-Jozsa Test Results:

Test: n=2, constant, mask=None
Expected: Should measure 00
Results: {'00': 1024}
Most frequent: 00
Correct: True

Test: n=2, constant, mask=1
Expected: Should measure 00
Results: {'00': 1024}
Most frequent: 00
Correct: True

Test: n=3, balanced, mask=101
Expected: Should measure 101
Results: {'101': 1024}
Most frequent: 101
Correct: True

Test: n=3, balanced, mask=111
Expected: Should measure 111
Results: {'111': 1024}
Most frequent: 111
Correct: True

Test: n=2, balanced, mask=01
Expected: Should measure 01
Results: {'01': 1024}
Most frequent: 01
Correct: True
